# Nuclio - prediction function

## Setup the environment

In [4]:
# nuclio: ignore
import nuclio

### Set environment variables

In [19]:
# Iguazio access
%nuclio env FRAMESD=${V3IO_FRAMESD}
%nuclio env V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

# Model handling
%nuclio env MODEL_FILE=lgbm.model
%nuclio env -c MODEL_FILEPATH=/tmp/mlmodel/${MODEL_FILE}
%nuclio env -l MODEL_FILEPATH=models/trained/${MODEL_FILE}

# Function variables
%nuclio env NUM_OF_PRODUCTS_TO_RETURN=4
%nuclio env CUSTOMERS_TABLE=recommendation_demo_customers
%nuclio env PREDIOCTION_TABLE=recommendation_demo_recommendations

%nuclio: setting 'FRAMESD' environment variable
%nuclio: setting 'V3IO_USERNAME' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'MODEL_FILE' environment variable
%nuclio: setting 'MODEL_FILEPATH' environment variable
%nuclio: setting 'NUM_OF_PRODUCTS_TO_RETURN' environment variable
%nuclio: setting 'CUSTOMERS_TABLE' environment variable
%nuclio: setting 'PREDIOCTION_TABLE' environment variable


### Installations

In [18]:
%%nuclio cmd  
pip install json
pip install requests
pip install pandas
pip install lightgbm
pip install v3io_frames

  Could not find a version that satisfies the requirement json (from versions: )
No matching distribution found for json


### Imports

In [20]:
# Util
import os
import json
import requests

# Function
import pandas as pd
import lightgbm as lgb

# DB
import v3io_frames as v3f

## Function code

### Init context

In [24]:
def init_context(context):
    # define DB
    client = v3f.Client(os.environ['FRAMESD'])
    setattr(context, 'client', client)
    
    # define Model
    model = lgb.Booster(model_file=os.environ['MODEL_FILEPATH'])
    setattr(context, 'model', model)
    
    # vars
    setattr(context, 'k', int(os.environ['NUM_OF_PRODUCTS_TO_RETURN']))

In [22]:
def handler(context, event):
    # Get user parameters
    df = client.read('kv', )

    # Predict
    df["prediction"] = context.model.predict(df)

    # Eliminate low confidence results
    df = df.loc[df.prediction > 0.01, ['prediction', 'product_id']]

    # Keep top products
    df.sort_values(by='prediction', ascending=False, inplace=True)
    best_products = df.iloc[:context.k, :]


    return json.dumps(best_products.to_json())

In [23]:
%nuclio show

%nuclio: notebook prediction exported
Config:
apiVersion: nuclio.io/v1
kind: Function
metadata:
  annotations:
    nuclio.io/generated_by: function generated at 15-04-2019 by iguazio from /User/tutorials/demos/location_based_recommendation/prediction.ipynb
  labels: {}
  name: prediction
spec:
  build:
    commands:
    - pip install json
    - pip install requests
    - pip install pandas
    - pip install lightgbm
    - pip install v3io_frames
    functionSourceCode: IyBHZW5lcmF0ZWQgYnkgbnVjbGlvLmV4cG9ydC5OdWNsaW9FeHBvcnRlciBvbiAyMDE5LTA0LTE1IDA3OjU2CgoKCgoKaW1wb3J0IG9zCmltcG9ydCBqc29uCmltcG9ydCByZXF1ZXN0cwoKaW1wb3J0IHBhbmRhcyBhcyBwZAppbXBvcnQgbGlnaHRnYm0gYXMgbGdiCgppbXBvcnQgdjNpb19mcmFtZXMgYXMgdjNmCgpkZWYgaW5pdF9jb250ZXh0KGNvbnRleHQpOgogICAgY2xpZW50ID0gdjNmLkNsaWVudChvcy5lbnZpcm9uWydGUkFNRVNEJ10pCiAgICBzZXRhdHRyKGNvbnRleHQsICdjbGllbnQnLCBjbGllbnQpCiAgICAKICAgIG1vZGVsID0gbGdiLkJvb3N0ZXIobW9kZWxfZmlsZT0ibGdiLm1vZGVsIikKICAgIHNldGF0dHIoY29udGV4dCwgJ21vZGVsJywgbW9kZWwpCiAgICAKICAgIHNldG